In [1]:
!pip install transformers==4.9.1 datasets wandb

     |████████████████████████████████| 2.6 MB 9.8 MB/s 
     |████████████████████████████████| 325 kB 59.8 MB/s 
     |████████████████████████████████| 1.7 MB 50.9 MB/s 
     |████████████████████████████████| 3.3 MB 45.8 MB/s 
     |████████████████████████████████| 895 kB 42.8 MB/s 
     |████████████████████████████████| 596 kB 51.2 MB/s 
     |████████████████████████████████| 1.1 MB 46.7 MB/s 
     |████████████████████████████████| 312 kB 43.5 MB/s 
     |████████████████████████████████| 311 kB 40.0 MB/s 
     |████████████████████████████████| 312 kB 50.4 MB/s 
     |████████████████████████████████| 311 kB 39.7 MB/s 
     |████████████████████████████████| 311 kB 37.4 MB/s 
     |████████████████████████████████| 306 kB 50.9 MB/s 
     |████████████████████████████████| 298 kB 46.9 MB/s 
     |████████████████████████████████| 298 kB 52.5 MB/s 
     |████████████████████████████████| 290 kB 63.3 MB/s 
     |████████████████████████████████| 290 kB 61.7 MB/s 
     |█████████

In [2]:
import tensorflow as tf
import torch, gc

gc.collect()
torch.cuda.empty_cache()

In [3]:
# GPU 메모리 에러 체크
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [4]:
# device type
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"# available GPUs : {torch.cuda.device_count()}")
    print(f"GPU name : {torch.cuda.get_device_name()}")
else:
    device = torch.device("cpu")
print(device)

# available GPUs : 1
GPU name : Tesla P100-PCIE-16GB
cuda


In [5]:
!huggingface-cli login


        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        
Username: valodiel@kakao.com
Password: 
Login successful
Your token: ZzxznowjVkJOJgNxLiuvfEwInLCiwPWlyrmyaKaSYgASEdIMDLMsQPiqFadwiutVUwoLcwXNrAGTdwmwUhgwsLEkaAQnrcupjggDoXnEpEOjPCJcxJwzdxFcyXvsfllP 

Your token has been saved to /root/.huggingface/token


In [6]:
!pip install hf-lfs
!git config --global user.email "esgsky1004@gmail.com"
!git config --global user.name "SeungguEum"

     |████████████████████████████████| 4.0 MB 7.4 MB/s 


In [7]:
import transformers
print(transformers.__version__)

4.9.1


In [8]:
# argment setting
task = "sts"
model_checkpoint = "klue/roberta-large"
batch_size = 32

In [9]:
from datasets import load_dataset
dataset = load_dataset('klue', task)

Downloading:   0%|          | 0.00/5.19k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.93k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset klue downloaded and prepared to /root/.cache/huggingface/datasets/klue/sts/1.0.0/55ff8f92b7a4b9842be6514ce0b4b5295b46d5e493f8bb5760da4be717018f90. Subsequent calls will reuse this data.


In [10]:
import re

In [11]:
def cleansing(text):
    text = re.sub('[0-9a-zA-Z-=+,#/\?:^$.@*\"※~&%ㆍ·!』\\‘〈〉|\(\)\[\]\<\>`\'…》《]','', text)
    return text

In [12]:
for i in range(len(dataset['train']['sentence1'])):
    dataset['train']['sentence1'][i] = cleansing(dataset['train']['sentence1'][i])
    dataset['train']['sentence2'][i] = cleansing(dataset['train']['sentence2'][i])

In [13]:
import torch
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/752k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [14]:
print(f"Sentence1: {dataset['train'][0]['sentence1']}\nSentence2: {dataset['train'][0]['sentence2']}")

Sentence1: 숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.
Sentence2: 숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.


In [15]:
dataset = dataset.flatten()
dataset = dataset.rename_column('labels.real-label','label')

In [16]:
max_length = 512

In [17]:
def preprocess_function(examples):
    return tokenizer(examples['sentence1'], examples['sentence2'],
                     truncation=True, max_length=max_length, padding=True)

In [18]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

  0%|          | 0/12 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [19]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
num_labels = 1
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint,
                                                           num_labels=num_labels)

Downloading:   0%|          | 0.00/547 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifi

In [20]:
metric_name = "pearsonr"

In [21]:
import os

model_name = model_checkpoint.split("/")[-1]
output_dir = os.path.join("test-klue", "sts")
logging_dir = os.path.join(output_dir, 'logs')
args = TrainingArguments(
    # checkpoint
    output_dir=output_dir,

    # Model Save & Load
    save_strategy = "epoch",
    load_best_model_at_end=True,

    # Dataset
    num_train_epochs=5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    
    # Optimizer
    learning_rate=2e-5, 
    weight_decay=0.01,  

    # Evaluation 
    metric_for_best_model='eval_' + metric_name,
    evaluation_strategy = "epoch",

    # HuggingFace Hub Upload
    push_to_hub=False,
    push_to_hub_model_id=f"{model_name}-finetuned-{task}",

    # Logging
    logging_dir=logging_dir,
    report_to='wandb',

    # Randomness
    seed=42,
)

In [22]:
# datasets 라이브러리에서 제공하는 Evaluation metric의 리스트를 확인합니다.
from datasets import list_metrics, load_metric
metrics_list = list_metrics()
len(metrics_list)
print(', '.join(metric for metric in metrics_list))

accuracy, bertscore, bleu, bleurt, cer, chrf, code_eval, comet, competition_math, coval, cuad, f1, frugalscore, gleu, glue, google_bleu, indic_glue, mae, mahalanobis, matthews_correlation, mauve, mean_iou, meteor, mse, pearsonr, perplexity, precision, recall, rouge, sacrebleu, sari, seqeval, spearmanr, squad, squad_v2, super_glue, ter, wer, wiki_split, xnli, xtreme_s


In [23]:
# STS의 metric은 pearsonr, F1 score을 사용합니다. [TODO] F1 metric 추가
metric_name = "pearsonr"
metric_pearsonr = load_metric(metric_name) # peason r
#metric_name2 = "f1"
#metric_f1 = load_metric(metric_name2) # F1

Downloading:   0%|          | 0.00/1.81k [00:00<?, ?B/s]

In [24]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions[:, 0]
    pr = metric_pearsonr.compute(predictions=predictions,
                                  references=labels)

    # f1 = metric_f1.compute(predictions=predictions,
    #                               references=labels)
    return pr

In [25]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset['validation'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [26]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [27]:
id = wandb.util.generate_id()
print(id)

1tw3bjjh


In [28]:
wandb.init(project="test-project", entity="preonboarding_team8")

# Optional
wandb.watch(model)

wandb: Currently logged in as: preonboarding_team8 (use `wandb login --relogin` to force relogin)


[]

In [29]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: sentence1, source, labels.label, labels.binary-label, guid, sentence2.
***** Running training *****
  Num examples = 11668
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 1825
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,Pearsonr
1,No log,0.472923,{'pearsonr': 0.8787315695292311}
2,0.402800,0.478724,{'pearsonr': 0.8855501963563255}


The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: sentence1, source, labels.label, labels.binary-label, guid, sentence2.
***** Running Evaluation *****
  Num examples = 519
  Batch size = 32
Saving model checkpoint to test-klue/sts/checkpoint-365
Configuration saved in test-klue/sts/checkpoint-365/config.json
Model weights saved in test-klue/sts/checkpoint-365/pytorch_model.bin
tokenizer config file saved in test-klue/sts/checkpoint-365/tokenizer_config.json
Special tokens file saved in test-klue/sts/checkpoint-365/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: sentence1, source, labels.label, labels.binary-label, guid, sentence2.
***** Running Evaluation *****
  Num examples = 519
  Batch size = 32
Saving model checkpoint to test-klue/sts/checkpoint-730
Configur

TypeError: ignored